In [4]:
# Imports

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [198]:
# Eight Queens State

class EightQueensState:
    def __init__(self, board_length, state=None):
        if state is None:
            self.state = np.random.randint(0, board_length, board_length)
        else:
            self.state = state
        self.goal_fitness = int(((board_length ** 2) - board_length) / 2)
        

    def fitness(self):
        count = 0
        n = len(self.state)

        # Check each queen against all queens to its right (0-6)
        for i in range(n - 1): 

            # Get remaining queens to check against
            remaining_queens = np.array(self.state[i + 1:])
            current_queen = self.state[i]

            # Count queens in same row to the right of current queen
            count += (current_queen == remaining_queens).sum()

            # Calculate diagonal positions
            upper_diagonal = current_queen + np.arange(1, n - i)
            lower_diagonal = current_queen - np.arange(1, n - i)
            
            # Count diagonal attacks
            count += (remaining_queens == upper_diagonal).sum()
            count += (remaining_queens == lower_diagonal).sum()

        fitness = self.goal_fitness - count  

        return fitness
    
    def get_state(self):
        return self.state

    def is_goal(self):
        if self.fitness() == self.goal_fitness:
            return 'Goal Fitness!', self.state, self.fitness()
        else:
            return None, None, self.fitness()
    

In [ ]:
class GeneticAlgorithm:
    def __init__(self, board_size, population_size, mutation_probability, pupulation_runs):
        self.fitness_history = []
        self.board_size = board_size
        self.population_size = population_size
        self.mutation_probability = mutation_probability
        self.pupulation_runs = pupulation_runs
        self.current_population = None


    def generate_population(self, init=False):

        if init:
            population = [EightQueensState(self.board_size) for _ in range(self.population_size)]
        else:
            population = [EightQueensState(self.board_size, self.current_population[population]) for population in range(len(self.current_population))]

        population_data = {}
        total_fitness = int(sum(individual.fitness() for individual in population))
        idx = 0

        for individual in population:
            idx += 1

            goal, state, fitness = individual.is_goal()
            self.fitness_history.append(fitness)

            if goal is not None:
                print(goal, state)
                return

            individual_fitness = individual.fitness()
            individual_state = individual.state

            population_data[idx] = {}
            population_data[idx]['fitness_pct'] = round((int(individual_fitness) / total_fitness) * 100, 1)
            population_data[idx]['state'] = individual_state

        population_data = dict(sorted(population_data.items(), key=lambda item: item[1]['fitness_pct'], reverse=True))

        return population_data
    


    def selection_mutation(self, population_data):

        new_popultaion = []

        for _ in range(int(len(population_data) / 2)):
                
            # Selection
            iterator = iter(population_data.items())

            individual_1 = next(iterator)[1]
            individual_2 = next(iterator)[1]

            selection_idx = np.random.randint(1, self.board_length)

            state_1 = individual_1['state']
            state_2 = individual_2['state']

            state_1_front = state_1[:selection_idx:]
            state_1_back = state_1[selection_idx:]
            state_2_front = state_2[:selection_idx]
            state_2_back = state_2[selection_idx:]

            # Crossover
            state_1_new = np.append(state_1_front, state_2_back)
            state_2_new = np.append(state_2_front, state_1_back)

            # Mutation
            def mutate(state):
                random_idx= np.random.choice(len(state))
                idx_range = list(set(range(0, self.board_length)) - {state[random_idx]})
                state[random_idx] = np.random.choice(idx_range)
                return state
            
            if np.random.rand() <= self.mutation_probability:
                state_1_new = mutate(state_1_new)

            if np.random.rand() <= self.mutation_probability:
                state_2_new = mutate(state_2_new)

            new_popultaion.append(state_1_new)
            new_popultaion.append(state_2_new)

            population_data = dict(list(population_data.items())[2:])

        return new_popultaion
    

    def run(self):

        for idx in range(self.pupulation_runs):

            if idx == 0:
                population_data = self.generate_population(init=True)
            else:
                population_data = self.generate_population()
                
            self.current_population = self.selection_mutation(population_data)

        return self.fitness_history


In [ ]:
GeneticAlgorithm(board_size=8, population_size=4, mutation_probability=0.1, pupulation_runs=100)


In [ ]:
# Other 8 queen functions 

    # @staticmethod
    # def copy_replace(state, i, x):
    #     new_state = state.copy()
    #     new_state[i] = x
    #     return new_state
    

    # @staticmethod
    # def range_missing(start, stop, missing):
    #     return list(range(start, missing)) + list(range(missing + 1, stop))
    


    # def neighbourhood(self):
    #     """This generates every state possible by changing a single queen position"""
    #     neighbourhood = []
    #     # Try moving each queen to every possible position in its column
    #     for column in range(self.n):
    #         # Get all valid row positions except current one
    #         for new_position in self.range_missing(0, self.n, self.state[column]):
    #             new_state = self.copy_replace(self.state, column, new_position)
    #             neighbourhood.append(EightQueensState(new_state))

    #     return neighbourhood


    # def random_neighbour(self):
    #     """Generates a single random neighbour state, useful for some algorithms"""
    #     # Pick random column and new row position
    #     column = np.random.choice(range(self.n))
    #     new_position = np.random.choice(self.range_missing(0, self.n, self.state[column]))
    #     new_state = self.copy_replace(self.state, column, new_position)

    #     return EightQueensState(new_state)


In [ ]:
# Hill search with plotting 

def find_solution_with_history():
    current_state = EightQueensState()
    steps = 0
    # Track more data for richer visualization
    history = [(0, current_state.calc_fitness(), current_state.state.copy(), "Initial State")]
    
    while not current_state.is_goal():
        next_state = min(current_state.neighbourhood(), key=lambda s: s.calc_fitness())
        
        if next_state.calc_fitness() >= current_state.calc_fitness():
            current_state = EightQueensState()
            history.append((steps, current_state.calc_fitness(), 
                          current_state.state.copy(), "Restart"))
        else:
            current_state = next_state
            steps += 1
            history.append((steps, current_state.calc_fitness(), 
                          current_state.state.copy(), "Progress"))
            print(current_state.calc_fitness())    

    # Add final solution
    history.append((steps + 1, current_state.calc_fitness(), 
                   current_state.state.copy(), "Solution"))
    
    return history

# Run algorithm and collect data
history = find_solution_with_history()

# Convert to DataFrame for easier plotting
df = pd.DataFrame(history, columns=['Step', 'Cost', 'State', 'Event'])


In [ ]:
# Plotting

fig = go.Figure()

# Add the main cost line
fig.add_trace(go.Scatter(
    x=df['Step'],
    y=df['Cost'],
    mode='lines+markers',
    name='Cost',
    line=dict(color='#00B5F7', width=2),
    marker=dict(size=6),
    hovertemplate="Step: %{x}<br>Cost: %{y}<br><extra></extra>"
))

# Add restart points with different color and size
restart_points = df[df['Event'] == 'Restart']
fig.add_trace(go.Scatter(
    x=restart_points['Step'],
    y=restart_points['Cost'],
    mode='markers',
    name='Restarts',
    marker=dict(
        color='#FF4444',
        size=12,
        symbol='diamond'
    ),
    hovertemplate="Restart at step: %{x}<br>Cost: %{y}<br><extra></extra>"
))

# Add the solution point
solution_point = df[df['Event'] == 'Solution']
fig.add_trace(go.Scatter(
    x=solution_point['Step'],
    y=solution_point['Cost'],
    mode='markers',
    name='Solution',
    marker=dict(
        color='#00FF00',
        size=15,
        symbol='star'
    ),
    hovertemplate="Solution found!<br>Step: %{x}<br>Cost: %{y}<br><extra></extra>"
))

# Update layout with dark theme
fig.update_layout(
    template='plotly_dark',
    plot_bgcolor='rgba(30, 30, 30, 1)',
    paper_bgcolor='rgba(30, 30, 30, 1)',
    title=dict(
        text='Eight Queens Puzzle - Cost Function Evolution',
        font=dict(size=24, color='white'),
        y=0.95
    ),
    xaxis=dict(
        title='Steps',
        gridcolor='rgba(128, 128, 128, 0.2)',
        zeroline=False,
        title_font=dict(size=16, color='white'),
    ),
    yaxis=dict(
        title='Cost (Number of Attacking Pairs)',
        gridcolor='rgba(128, 128, 128, 0.2)',
        zeroline=False,
        title_font=dict(size=16, color='white'),
    ),
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor='rgba(50, 50, 50, 0.8)',
        font=dict(color='white')
    ),
    hovermode='x unified',
    margin=dict(t=100, b=50, l=50, r=50)
)

# Add annotations for key events
fig.add_annotation(
    x=df.iloc[-1]['Step'],
    y=df.iloc[-1]['Cost'],
    text="Solution Found!",
    showarrow=True,
    arrowhead=1,
    ax=50,
    ay=-50,
    font=dict(size=14, color='#00FF00'),
    bgcolor='rgba(50, 50, 50, 0.8)'
)

fig.show()

In [50]:
# Genetic Algorithm - Russell and Norvig aligned

class GeneticEightQueens:
    def __init__(self, population_size=50, mutation_rate=0.1):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.n = 8
        
    def _calculate_fitness(self, population):
        """Vectorized fitness calculation for entire population"""
        fitness_scores = np.zeros(len(population))
        
        for i, state in enumerate(population):
            conflicts = 0
            for j in range(self.n - 1):
                # Row conflicts
                conflicts += np.sum(state[j] == state[j + 1:])
                
                # Diagonal conflicts
                diag_up = state[j] + np.arange(1, self.n - j)
                diag_down = state[j] - np.arange(1, self.n - j)
                conflicts += np.sum(state[j + 1:] == diag_up)
                conflicts += np.sum(state[j + 1:] == diag_down)
                
            fitness_scores[i] = 28 - conflicts
            
        return fitness_scores
    
    def _tournament_select(self, population, fitness_scores):
        """Simple tournament selection"""
        idx = np.random.randint(0, len(population), 2)
        if fitness_scores[idx[0]] > fitness_scores[idx[1]]:
            return population[idx[0]]
        return population[idx[1]]
    
    def solve(self, max_generations=1000):
        # Initialize population
        population = np.random.randint(0, self.n, size=(self.population_size, self.n))
        best_fitness_history = []
        
        for generation in range(max_generations):
            # Calculate fitness for all individuals
            fitness_scores = self._calculate_fitness(population)
            best_fitness = np.max(fitness_scores)
            best_fitness_history.append(best_fitness)
            
            # Check if solution found
            if best_fitness == 28:
                best_idx = np.argmax(fitness_scores)
                return EightQueensState(population[best_idx]), best_fitness_history
            
            # Create new population
            new_population = np.zeros((self.population_size, self.n), dtype=int)
            
            # Elitism - keep best individual
            best_idx = np.argmax(fitness_scores)
            new_population[0] = population[best_idx]
            
            # Generate rest of new population
            for i in range(1, self.population_size):
                # Select parents
                parent1 = self._tournament_select(population, fitness_scores)
                parent2 = self._tournament_select(population, fitness_scores)
                
                # Crossover
                crossover_point = np.random.randint(1, self.n)
                child = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
                
                # Mutation
                if np.random.random() < self.mutation_rate:
                    mutation_point = np.random.randint(0, self.n)
                    new_value = np.random.randint(0, self.n)
                    while new_value == child[mutation_point]:
                        new_value = np.random.randint(0, self.n)
                    child[mutation_point] = new_value
                
                new_population[i] = child
            
            population = new_population
        
        # Return best solution found
        best_idx = np.argmax(self._calculate_fitness(population))
        return EightQueensState(population[best_idx]), best_fitness_history

In [48]:
# Plot GA

def plot_ga(solution, fitness_history):

    # Create DataFrame
    df = pd.DataFrame({
        'Generation': range(len(fitness_history)),
        'Fitness': fitness_history
    })

    # Create the figure with dark theme using plotly express
    fig = px.line(
        df,
        x='Generation',
        y='Fitness',
        title='Eight Queens - Genetic Algorithm Fitness Evolution',
        labels={'Fitness': 'Fitness Score (28 = Perfect Solution)', 'Generation': 'Generation'},
        template='plotly_dark'
    )

    # Add the solution point as a marker if found
    solution_point = df[df['Fitness'] == 28]
    if not solution_point.empty:
        fig.add_scatter(
            x=solution_point['Generation'],
            y=solution_point['Fitness'],
            mode='markers',
            name='Solution Found',
            marker=dict(color='#00FF00', size=15, symbol='star')
        )

    # Add reference line for perfect fitness
    fig.add_hline(
        y=28, 
        line_dash="dash", 
        line_color="rgba(255, 255, 255, 0.3)",
        annotation_text="Perfect Fitness (28)",
        annotation_position="right",
        annotation_font_color="white"
    )

    # Add annotations for initial and final fitness
    fig.add_annotation(
        x=0,
        y=fitness_history[0],
        text=f"Initial Fitness: {fitness_history[0]}",
        showarrow=True,
        arrowhead=1,
        ax=-50,
        ay=-30,
        font=dict(size=14, color='#00B5F7'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    if solution_point.empty:
        final_gen = len(fitness_history) - 1
        final_fitness = fitness_history[-1]
        annotation_text = f"Best Fitness: {final_fitness}"
    else:
        final_gen = solution_point['Generation'].iloc[0]
        annotation_text = "Solution Found!"

    fig.add_annotation(
        x=final_gen,
        y=fitness_history[final_gen],
        text=annotation_text,
        showarrow=True,
        arrowhead=1,
        ax=50,
        ay=-50,
        font=dict(size=14, color='#00FF00'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    # Final adjustments
    fig.update_layout(
        xaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False),
        yaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False, range=[min(fitness_history) - 1, 29]),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor='rgba(50, 50, 50, 0.8)'
        ),
        hovermode='x unified',
        margin=dict(t=100, b=50, l=50, r=50)
    )

    print(f"Solution found in {final_gen + 1} generations")
    print(f"Final fitness: {fitness_history[final_gen]}")
    fig.show()

In [ ]:
# Run the genetic algorithm and collect data

ga = GeneticEightQueens(population_size=8, mutation_rate=0.1)
solution, fitness_history = ga.solve(max_generations=1000)
plot_ga(solution, fitness_history)